# Desenvolvendo a Dimensão Cliente 

scd1

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [3]:
# importando 

import pandas as pd 
from sqlalchemy import text
from sqlalchemy import create_engine
from dotenv import load_dotenv
from utils.acessobanco import acessobanco

In [4]:
#Carrega as variáveis de ambiente do arquivo .env

load_dotenv(dotenv_path="/home/danielpedro/Python/Projetos/SuperStorePY/Bancos/acessobanco.env")


True

In [6]:
# Cria a conexão com o banco usando a função acessobanco()

engine = acessobanco()

In [7]:
# Criando a df puxando do banco

df = pd.read_sql_table("orders",engine, schema="stg")

In [8]:
# Verificando os tipos dos dados

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             9994 non-null   object
 1   id_pedido      9994 non-null   object
 2   data_pedido    9994 non-null   object
 3   data_envio     9994 non-null   object
 4   modo_envio     9994 non-null   object
 5   id_cliente     9994 non-null   object
 6   nome_cliente   9994 non-null   object
 7   segmento       9994 non-null   object
 8   pais           9994 non-null   object
 9   cidade         9994 non-null   object
 10  estado         9994 non-null   object
 11  ce             9994 non-null   object
 12  regiao         9994 non-null   object
 13  id_produto     9994 non-null   object
 14  categoria      9994 non-null   object
 15  sub_categoria  9994 non-null   object
 16  nome_produto   9994 non-null   object
 17  valor_venda    9994 non-null   object
 18  quantidade     9994 non-null

In [9]:
# Definido as colunas de dim cliente

df_DimCliente = df[['id_cliente','nome_cliente','segmento','pais','cidade','regiao','ce']]

In [10]:
# Aqui podemos perceber que a tabela contem duplicados pois estou puxando apenas as colunas da
# fato e não realizando um select destinct

df_DimCliente.duplicated().value_counts()

True     5084
False    4910
Name: count, dtype: int64

In [11]:
# Desta forma consigo limpar os dados duplicados usando como referencia id_cliente
# O subset=['Id_Cliente'], ele olha só essa coluna para decidir o que é duplicado.

df_DimCliente = df_DimCliente.drop_duplicates(subset=['id_cliente'])

In [12]:
# Realizando uma verificação de duplicados
 
df_DimCliente.duplicated().value_counts()

False    793
Name: count, dtype: int64

In [13]:
# Ordenando os dados por 'nome_cliente'

df_DimCliente =  df_DimCliente.sort_values(by=['nome_cliente'], ascending=True)
df_DimCliente.head(10)

,id_cliente,nome_cliente,segmento,pais,cidade,regiao,ce
3938,AB-10015,Aaron Bergman,Consumer,United States,Oklahoma City,Central,73120
157,AH-10030,Aaron Hawkins,Corporate,United States,Philadelphia,East,19134
1177,AS-10045,Aaron Smayling,Corporate,United States,Pasadena,West,91104
829,AB-10060,Adam Bellavance,Home Office,United States,Seattle,West,98105
509,AH-10075,Adam Hart,Corporate,United States,Atlanta,South,30318
398,AS-10090,Adam Shillingsburg,Consumer,United States,Charlottesville,South,22901
619,AB-10105,Adrian Barton,Consumer,United States,Henderson,South,42420
922,AH-10120,Adrian Hane,Home Office,United States,Dallas,Central,75217
610,AS-10135,Adrian Shami,Home Office,United States,New York City,East,10035
613,AB-10150,Aimee Bixby,Consumer,United States,Carrollton,Central,75007


In [14]:
# Inserindo um valor para testes
#novo_cliente = {
#    "id_cliente": 999,
#    "nome_cliente": "Daniel Pedro",
#    "segmento": "Varejo",
#    "pais": "Brasil",
#    "cidade": "Paulista",
#    "regiao": "Nordeste",
#    "ce": "53400000"
#}


In [15]:
# df_DimCliente = pd.concat([df_DimCliente, pd.DataFrame([novo_cliente])], ignore_index=True)

In [16]:
# Criando uma tabela temporaria para aplicar o scd1

with engine.begin() as conn:
    df_DimCliente.to_sql(
        "tmp_dim_cliente",  # nome da tabela temporária no schema dw
        conn,               # conexão com o banco de dados
        schema="dw",        # schema onde a tabela será criada
        if_exists="replace", # substitui a tabela se já existir (garante dados atualizados)
        index=False,         # não cria coluna de índice do pandas
        method="multi",      # insere várias linhas por comando INSERT (melhor performance)
        chunksize=5000       # envia os dados em blocos de 5000 linhas (controle de memória)
    )



In [17]:
# Verificando a quantidade de colunas que subiram no temp

count = pd.read_sql("SELECT COUNT(*) as total_registros FROM dw.tmp_dim_cliente", engine).iloc[0,0]
print(f"Total de registros na tabela: {count}")

Total de registros na tabela: 793


In [18]:
# Atualiza registros existentes em batch com base em id_cliente

update_sql = """
UPDATE dw.dim_cliente_scd1 AS d
SET 
    nome_cliente = t.nome_cliente,
    segmento = t.segmento,
    pais = t.pais,
    cidade = t.cidade,
    regiao = t.regiao,
    ce = t.ce
FROM dw.tmp_dim_cliente AS t
WHERE d.id_cliente = t.id_cliente
  AND (
        d.nome_cliente IS DISTINCT FROM t.nome_cliente OR
        d.segmento IS DISTINCT FROM t.segmento OR
        d.pais IS DISTINCT FROM t.pais OR
        d.cidade IS DISTINCT FROM t.cidade OR
        d.regiao IS DISTINCT FROM t.regiao OR
        d.ce IS DISTINCT FROM t.ce
      );
"""

with engine.begin() as conn:
    conn.execute(text(update_sql))


In [19]:
# Insere apenas clientes novos

insert_sql = """
INSERT INTO dw.dim_cliente_scd1 (id_cliente, nome_cliente, segmento, pais, cidade, regiao, ce)
SELECT t.id_cliente, t.nome_cliente, t.segmento, t.pais, t.cidade, t.regiao, t.ce
FROM dw.tmp_dim_cliente AS t
LEFT JOIN dw.dim_cliente_scd1 AS d ON d.id_cliente = t.id_cliente
WHERE d.id_cliente IS NULL;
"""

with engine.begin() as conn:
    result = conn.execute(text(insert_sql))

print(f"Linhas inseridas: {result.rowcount}")

Linhas inseridas: 0
